do pliku `build.gradle(Module)`

```kotlin
dependencies {

    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.2")

    ...
}
```

```kotlin
// =================================================================================
// --- Główna Aktywność ---
// =================================================================================
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            StateFlowBasicsTheme {
                StateFlowDemoScreen()
            }
        }
    }
}

// =================================================================================
// --- ViewModel jako "Streamer" ---
// =================================================================================

class HotStreamViewModel : ViewModel() {
    // Prywatny, modyfikowalny StateFlow - licznik, który widzi tylko streamer.
    private val _numberStream = MutableStateFlow(0)
    // Publiczny, niemodyfikowalny StateFlow - to widzi publiczność.
    val numberStream: StateFlow<Int> = _numberStream.asStateFlow()

    init {
        // Streamer rozpoczyna "transmisję na żywo" w momencie utworzenia ViewModelu.
        // Strumień jest "gorący" - działa niezależnie od liczby widzów (subskrybentów).
        viewModelScope.launch {
            var count = 0
            while (true) {
                delay(1000)
                count++
                _numberStream.value = count
            }
        }
    }
}

// =================================================================================
// --- Ekran Demonstracyjny ---
// =================================================================================

@Composable
fun StateFlowDemoScreen(viewModel: HotStreamViewModel = viewModel()) {
    val liveValue by viewModel.numberStream.collectAsStateWithLifecycle()

    // Stany dla dwóch niezależnych widzów
    val scope = rememberCoroutineScope()
    var viewer1Value by remember { mutableStateOf("Nie subskrybuje") }
    var viewer1Job by remember { mutableStateOf<Job?>(null) }
    var viewer2Value by remember { mutableStateOf("Nie subskrybuje") }
    var viewer2Job by remember { mutableStateOf<Job?>(null) }

    Column(
        modifier = Modifier.fillMaxSize().padding(16.dp),
        horizontalAlignment = Alignment.CenterHorizontally
    ) {
        Text(
            "Demonstracja 'Gorącego' StateFlow",
            style = MaterialTheme.typography.headlineMedium,
            textAlign = TextAlign.Center
        )
        Text("(Transmisja na żywo)")

        Card(modifier = Modifier.padding(vertical = 16.dp)) {
            Text(
                text = "Streamer nadaje: $liveValue",
                fontSize = 24.sp,
                fontWeight = FontWeight.Bold,
                modifier = Modifier.padding(16.dp)
            )
        }

        // Panel dla Widza 1
        ViewerPanel(
            viewerName = "Widz 1",
            status = viewer1Value,
            isSubscribed = viewer1Job?.isActive == true,
            onSubscribe = {
                viewer1Job = scope.launch {
                    viewModel.numberStream.collect { number ->
                        viewer1Value = "Odbiera: $number"
                    }
                }
            },
            onUnsubscribe = {
                viewer1Job?.cancel()
                viewer1Value = "Nie subskrybuje"
            }
        )

        // Panel dla Widza 2
        ViewerPanel(
            viewerName = "Widz 2",
            status = viewer2Value,
            isSubscribed = viewer2Job?.isActive == true,
            onSubscribe = {
                viewer2Job = scope.launch {
                    viewModel.numberStream.collect { number ->
                        viewer2Value = "Odbiera: $number"
                    }
                }
            },
            onUnsubscribe = {
                viewer2Job?.cancel()
                viewer2Value = "Nie subskrybuje"
            }
        )
    }
}

@Composable
private fun ViewerPanel(
    viewerName: String,
    status: String,
    isSubscribed: Boolean,
    onSubscribe: () -> Unit,
    onUnsubscribe: () -> Unit
) {
    Column(
        modifier = Modifier.fillMaxWidth().padding(vertical = 8.dp),
        horizontalAlignment = Alignment.CenterHorizontally
    ) {
        Text(status, style = MaterialTheme.typography.titleMedium)
        Row(horizontalArrangement = Arrangement.Center) {
            Button(onClick = onSubscribe, enabled = !isSubscribed) {
                Text("Subskrybuj ($viewerName)")
            }
            Spacer(Modifier.width(8.dp))
            Button(onClick = onUnsubscribe, enabled = isSubscribed) {
                Text("Anuluj subskrypcję")
            }
        }
        HorizontalDivider(
            modifier = Modifier.padding(top = 16.dp),
            thickness = DividerDefaults.Thickness,
            color = DividerDefaults.color
        )
    }
}
```